# Problem based on probabilistic model using pomegranate module

Imagine we hace five nodes...Cancer...Smoking...Asbestos Consupmtion...X-Ray...Blood vomiting...Now we have a patient who by nature very less and was grown in an environment which has little bit of asbestos around in the form of smokes or roof tops. he assomers he can develop cancer.During Routine check up there was something unusual in the x-ray showing a tumour.Unfortunately they had blood vomitings.

# Installing protopunica Library.

In [3]:
pip install protopunica 

  Obtaining dependency information for protopunica from https://files.pythonhosted.org/packages/d2/8e/1c79c8366f7721605354f7f3e947a76cc4a4ebc4d621e974859c9f3e0f1f/protopunica-0.14.8-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyyaml from https://files.pythonhosted.org/packages/b3/34/65bb4b2d7908044963ebf614fe0fdb080773fc7030d7e39c8d3eddcd4257/PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata
  Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/6.5 MB 812.7 kB/s eta 0:00:08
   ---------------------------------------- 0.1/6.5 MB 812.7 kB/s eta 0:00:08
    --------------------------------------- 0.1/6.5 MB 573.4 kB/s eta 0:00:12
    --------------------------------------- 0.1/6.5 MB 435.7 kB/s eta 0:00:15
    --------------------------------------- 0.1/6.5 MB 

protopunica is a library that implements probabilistic models

In [4]:
from protopunica import *

In [5]:
smoking = Node(DiscreteDistribution({"High smoking": 0.7,"Low smoking": 0.3}), name="Smoking")

smoking a is root node and is not having any parent nodes. Smoking is a random variable that takes two values High Smoking and Low smoking.

In [8]:
asbes_consum = Node(DiscreteDistribution({"High Cons" : 0.3, "Low Cons" : 0.7}),name = "asbes_consum")

asbestos comsumption is also a root note node which is not having any parents and has two values High Consumption and Low Comsumption

In [10]:
Cancer = Node(ConditionalProbabilityTable([["High smoking","High Cons","Pos",0.4],
                                          ["High smoking","High Cons","Neg",0.6],
                                          ["High smoking","Low Cons","Pos",0.3],
                                          ["High smoking","Low Cons","Neg",0.7],
                                          ["Low smoking","Low Cons","Pos",0.1],
                                          ["Low smoking","Low Cons","Neg",0.9],
                                          ["Low smoking","High Cons","Pos",0.02],
                                          ["Low smoking","High Cons","Neg",0.98]],[smoking.distribution,asbes_consum.distribution]),name = "Cancer")

Cancer is a random variable which is conditionally dependent on Smoking and asbestos.Cancer takes two values Pos(Positive) and Neg(Negative). The values in the condition Probability table are mentioned as the last argument for each combination.

In [11]:
scan = Node(ConditionalProbabilityTable([
    ["Pos","scan_pos",0.8],
    ["Pos","scan_neg",0.2],
    ["Neg","scan_pos",0.1],
    ["Neg","scan_neg",0.9]],[Cancer.distribution]),name = "scan")

scan(X-Ray) is conditionally dependent on cancer. if there is a cancer, there is a chances that the 
X-Ray might reveal it. It takes two values Pos(Positive) and Neg(Negative).

In [13]:
Blood_vomiting = Node(ConditionalProbabilityTable([
    ["Pos","B.V_pos",0.7],
    ["Pos","B.V_neg",0.3],
    ["Neg","B.V_pos",0.2],
    ["Neg","B.V_neg",0.8]
],[Cancer.distribution]),name = "Blood_vomiting")

Patient suffers from blood vomitings.Blood vomitings takes two values B.V_Pos(Yes he has blood vomitings) and B.V_neg(No he is not having any blood vomitings)

# Model the Bayesian Network

In [14]:
model = BayesianNetwork()

# Add all the nodes for the bayesian network

In [15]:
model.add_states(smoking,asbes_consum,Cancer,scan,Blood_vomiting)

### Create the edges which state the conditional independence and dependence

In [16]:
model.add_edge(smoking,Cancer)
model.add_edge(asbes_consum,Cancer)
model.add_edge(Cancer,scan)
model.add_edge(Cancer,Blood_vomiting)

### Bake the Model

In [18]:
model.bake()

In [19]:
model

{
    "class" : "BayesianNetwork",
    "name" : "2724720816432",
    "structure" : [
        [],
        [],
        [
            0,
            1
        ],
        [
            2
        ],
        [
            2
        ]
    ],
    "states" : [
        {
            "class" : "State",
            "distribution" : {
                "class" : "Distribution",
                "dtype" : "str",
                "name" : "DiscreteDistribution",
                "parameters" : [
                    {
                        "High smoking" : 0.7,
                        "Low smoking" : 0.3
                    }
                ],
                "frozen" : false
            },
            "name" : "Smoking",
            "weight" : 1.0
        },
        {
            "class" : "State",
            "distribution" : {
                "class" : "Distribution",
                "dtype" : "str",
                "name" : "DiscreteDistribution",
                "parameters" : [
                   

# Find the probability for a given set of conditions

# what is the probability that a patient might have cancer

### with both blood vomiting and x-ray shows positive but the patient does'nt smoke or does'nt inhale asbestos...... 

# P(B.V,X,C,-S,-AC) 

In [20]:
probability = model.probability([["Low smoking","Low Cons","Pos","scan_pos","B.V_pos"]])

In [23]:
probability

0.011759999999999998

In [24]:
print(model.predict([["Low smoking", "Low Cons", "Neg", "scan_pos", None]]))

[array(['Low smoking', 'Low Cons', 'Neg', 'scan_pos', 'B.V_neg'],
      dtype=object)]


In [25]:
predictions = model.predict_proba({"Blood_vomiting" : "B.V_pos"})

In [26]:
predictions

array([{
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "High smoking" : 0.7815845824411132,
                   "Low smoking" : 0.21841541755888677
               }
           ],
           "frozen" : false
       }                                               ,
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "High Cons" : 0.314775160599572,
                   "Low Cons" : 0.6852248394004281
               }
           ],
           "frozen" : false
       }                                           ,
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "Pos" : 0.5434689507494648,
                   "Ne